# CE7455 Assignment 1
Peng Hongyi (G2105029E)

## Run the provided code at first

### Load data

In [28]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')

In [29]:
import data 

In [30]:
data_dir = './data/wikitext-2'
corpus = data.Corpus(data_dir)
print(f'Train: {corpus.train.shape}, Val: {corpus.valid.shape}, Test: {corpus.test.shape}')

Train: torch.Size([2088628]), Val: torch.Size([217646]), Test: torch.Size([245569])


In [31]:
def batchify(data, bsz):
    # Work out how cleanly we can divide the dataset into bsz parts.
    nbatch = data.size(0) // bsz
    # Trim off any extra elements that wouldn't cleanly fit (remainders).
    data = data.narrow(0, 0, nbatch * bsz)
    # Evenly divide the data across the bsz batches.
    data = data.view(bsz, -1).t().contiguous()
    return data.to(device)

In [32]:
BATCH_SIZE = 20
EVAL_BATCH_SIZE = 10
train_data = batchify(corpus.train, BATCH_SIZE)
val_data = batchify(corpus.valid, EVAL_BATCH_SIZE)
test_data = batchify(corpus.test, EVAL_BATCH_SIZE)

In [39]:
ntokens = len(corpus.dictionary)
import model
MODEL = "LSTM"
EMSIZE = 200
N_HID = 200
N_LAYERS = 2
DROPOUT = 0.2
TIED = "store_true"
model = model.RNNModel(MODEL, ntokens, EMSIZE, N_HID, N_LAYERS, DROPOUT, TIED).to(device)

In [42]:
BPTT = 3
def get_batch(source, i):
    seq_len = min(BPTT, len(source) - 1 - i)
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len].view(-1)
    return data, target

In [43]:
get_batch(train_data, 1)

(tensor([[    1,   357,    43,  2977,   530, 23080,    13,    78,    17,     0,
           4312,     0,   151,    22, 18215,    17,    17,    46,    43,  2015],
         [    2,  1496,  7369,   115,  4782,    37, 22196,   252, 26998,     0,
          28680,     1,   496,  2193,  1037,     9,  4072,   380,    27, 33001],
         [    3,   449,   310,     9,    13,  8034,  3107,   639,    13, 27958,
            638,     1,   168,    17,    43,  2786,    15,   160,   152,  3072]],
        device='cuda:0'),
 tensor([    2,  1496,  7369,   115,  4782,    37, 22196,   252, 26998,     0,
         28680,     1,   496,  2193,  1037,     9,  4072,   380,    27, 33001,
             3,   449,   310,     9,    13,  8034,  3107,   639,    13, 27958,
           638,     1,   168,    17,    43,  2786,    15,   160,   152,  3072,
             4,  5181, 15182, 18712,   877,    16,   423,    22,   562,  1575,
           496,     1,   209,  1056,    17,    39,   317, 19914,   128,   348],
        device=

In [ ]:
from torch import nn
criterion = nn.NLLLoss()
